<h1>Deep Learning - Processing</h1>

<h5>Data Parameters</h5>

In [ ]:
try:
    from google.colab import output
    IN_COLAB = True
except:
    output = None
    IN_COLAB = False

if IN_COLAB:
    datasets_folder = '/drive/My Drive/Colab Notebooks/DataSets/'
    work_folder = '/drive/My Drive/Colab Notebooks/Experiments/Deep Learning/'
else:
    datasets_folder = '/Google Drive/Colab Notebooks/DataSets/'
    work_folder = '/Google Drive/Colab Notebooks/Experiments/Deep Learning/'

print("In Colab:", IN_COLAB)
print("Work Folder:", work_folder)

<h5>Importing Packages</h5>

In [ ]:
try:
    import livelossplot
except:
    !pip install livelossplot --quiet

In [ ]:
import os
import gc
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import accuracy_score, plot_confusion_matrix, confusion_matrix, classification_report
import seaborn as sns
# pd.set_option("float_format", '{:0.10f}'.format)
# pd.set_option('display.max_columns', 30) 

In [ ]:
# -------------------------------------
# TensorFlow/Keras Selection
# -------------------------------------

# useKerasTfV1=False
# useKerasTfV2=False
# useTfV2=True

if useKerasTfV1:

    if IN_COLAB:
        %tensorflow_version 1.x
    
    import tensorflow as tf
    from tensorflow.python.util import deprecation
    deprecation._PRINT_DEPRECATION_WARNINGS = False
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    tf.logging.set_verbosity(tf.logging.ERROR)
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    
    import keras
    from keras.models import Sequential
    from keras.callbacks import History, ModelCheckpoint, CSVLogger, EarlyStopping
    from keras.layers import Activation, Dense, Dropout, SpatialDropout1D, Conv1D, TimeDistributed, MaxPooling1D, Flatten, ConvLSTM2D, Bidirectional, BatchNormalization, GlobalAvgPool1D, GlobalAveragePooling1D, MaxPooling1D, CuDNNLSTM as LSTM
    from keras.utils import plot_model
    
    from livelossplot import PlotLossesKeras

    print("Using Tensorflow", tf.__version__)
    print("Using Keras", keras.__version__)

if useKerasTfV2:
    
    import tensorflow as tf
    
    import keras
    from keras.models import Sequential
    from keras.callbacks import History, ModelCheckpoint, CSVLogger, EarlyStopping
    from keras.layers import Activation, Dense, Dropout, SpatialDropout1D, Conv1D, TimeDistributed, MaxPooling1D, Flatten, ConvLSTM2D, Bidirectional, BatchNormalization, GlobalAvgPool1D, GlobalAveragePooling1D, MaxPooling1D, LSTM
    from keras.utils import plot_model
    
    from livelossplot import PlotLossesKerasTF
    
    print("Using Tensorflow", tf.__version__)
    print("Using Keras", keras.__version__)

    if not(keras.__version__.startswith("2.3")):
        !pip install "keras>=2.3.0"

if useTfV2:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.callbacks import History, ModelCheckpoint, CSVLogger, EarlyStopping
    from tensorflow.keras.layers import Activation, Dense, Dropout, SpatialDropout1D, Conv1D, TimeDistributed, MaxPooling1D, Flatten, ConvLSTM2D, Bidirectional, BatchNormalization, GlobalAvgPool1D, GlobalAveragePooling1D, MaxPooling1D, LSTM
    from tensorflow.keras.utils import plot_model
    
    from livelossplot import PlotLossesKerasTF
    
    print("Using Tensorflow", tf.__version__)

In [ ]:
# print("Devices Tensorflow")
# print(tf.config.list_physical_devices())
# print(tf.compat.v1.config.list_physical_devices())
# from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())
# print("Devices Nvidia")
# !nvidia-smi

<h5>Data Functions</h5>

In [ ]:
# Load raw datasets from disk
# Input: folder where is the datasets folder and files
# Output: dict -> { "pvs_x": { "left": df, "right": df, "labels": df } }
def getDataSets(folder=datasets_folder):
    
    datasets = {}
    
    for i in range(1, 10):
        
        dataset_folder = os.path.join(folder, "PVS " + str(i))
        
        left =   pd.read_csv(os.path.join(dataset_folder, 'dataset_gps_mpu_left.csv'),  float_precision="high")
        right =  pd.read_csv(os.path.join(dataset_folder, 'dataset_gps_mpu_right.csv'), float_precision="high")
        labels = pd.read_csv(os.path.join(dataset_folder, 'dataset_labels.csv'),        float_precision="high")
        
        datasets["pvs_" + str(i)] = {
            "left": left,
            "right": right,
            "labels": labels
        }
    
    return datasets

# Get fields filtering by inputs
# Input: data types and placements
# Output: string[]
def getFields(acc=False, gyro=False, mag=False, temp=False, speed=False, location=False, below_suspension=False, above_suspension=False, dashboard=False):
    
    all_fields = [
        'timestamp', 
        'acc_x_dashboard', 'acc_y_dashboard', 'acc_z_dashboard',
        'acc_x_above_suspension', 'acc_y_above_suspension', 'acc_z_above_suspension', 
        'acc_x_below_suspension', 'acc_y_below_suspension', 'acc_z_below_suspension', 
        'gyro_x_dashboard', 'gyro_y_dashboard', 'gyro_z_dashboard', 
        'gyro_x_above_suspension', 'gyro_y_above_suspension', 'gyro_z_above_suspension',
        'gyro_x_below_suspension', 'gyro_y_below_suspension', 'gyro_z_below_suspension', 
        'mag_x_dashboard', 'mag_y_dashboard', 'mag_z_dashboard', 
        'mag_x_above_suspension', 'mag_y_above_suspension', 'mag_z_above_suspension', 
        'temp_dashboard', 'temp_above_suspension', 'temp_below_suspension', 
        'timestamp_gps', 'latitude', 'longitude', 'speed'
    ]
    
    return_fields = []
    
    for field in all_fields:
            
        data_type = False
        placement = False
        
        if(speed and field == "speed"):
            placement = data_type = True
            
        if(location and (field == "latitude" or field == "longitude")):
            placement = data_type = True
        
        if(acc):
            data_type = data_type or field.startswith("acc_")
        
        if(gyro):
            data_type = data_type or field.startswith("gyro_")
            
        if(mag):
            data_type = data_type or field.startswith("mag_")
            
        if(temp):
            data_type = data_type or field.startswith("temp_")
            
        if(below_suspension):
            placement = placement or field.endswith("_below_suspension")
            
        if(above_suspension):
            placement = placement or field.endswith("_above_suspension")
            
        if(dashboard):
            placement = placement or field.endswith("_dashboard")
        
        if(data_type and placement):
            return_fields.append(field)
            
    return return_fields

# Get subsets from raw datasets. 
# For each raw dataset, returns a subset with only fields passed.
# Input: raw datasets (dict), fields (string[]) and labels (string[])
# Output: dict -> { "pvs_x": { "left": df, "right": df, "labels": df } }
def getSubSets(datasets, fields, labels):
    
    subsets = {}
    
    for key in datasets.keys():
        
        subsets[key] = {
            "left": datasets[key]["left"][fields],
            "right": datasets[key]["right"][fields],
            "labels": datasets[key]["labels"][labels]
        }
    
    return subsets

# Get normalized data.
# Input: subsets (dict).
# Output: dict -> { "pvs_x": { "left": df, "right": df, "labels": df } }
def getNormalizedDataMinMax(subsets, scaler_range):
    
    normalized_sets = {}
    learn_data = pd.DataFrame()

    for pvs in subsets.keys():
        for side in ["left", "right"]:
            learn_data = learn_data.append(subsets[pvs][side], ignore_index=True)
        
    # Normalize between scaler_min and scaler_max
    scaler = MinMaxScaler(feature_range=scaler_range)
    scaler.fit(learn_data)
    
    for pvs in subsets.keys():
        
        normalized_sets[pvs] = {
            'left':  pd.DataFrame(data=scaler.transform(subsets[pvs]['left']),  columns=subsets[pvs]['left'].columns),
            'right': pd.DataFrame(data=scaler.transform(subsets[pvs]['right']), columns=subsets[pvs]['right'].columns),
            'labels': subsets[pvs]['labels']
        }
                    
    return scaler, normalized_sets 

# Get standardized data.
# Input: subsets (dict).
# Output: dict -> { "pvs_x": { "left": df, "right": df, "labels": df } }
def getNormalizedDataRobust(subsets):
    
    normalized_sets = {}
    learn_data = pd.DataFrame()

    for pvs in subsets.keys():
        for side in ["left", "right"]:
            learn_data = learn_data.append(subsets[pvs][side], ignore_index=True)
        
    # Standardize
    scaler = RobustScaler()
    scaler = scaler.fit(learn_data)
    
    for pvs in subsets.keys():
        
        normalized_sets[pvs] = {
            'left':  pd.DataFrame(data=scaler.transform(subsets[pvs]['left']),  columns=subsets[pvs]['left'].columns),
            'right': pd.DataFrame(data=scaler.transform(subsets[pvs]['right']), columns=subsets[pvs]['right'].columns),
            'labels': subsets[pvs]['labels']
        }
                    
    return scaler, normalized_sets 

# Inputs and outputs -> np.array with lines x columns (n samples x values/variables/features).
# moving_window: if window is moving
# shape: (None, ..., ..., n variables)
# Mode label: most common value in window, else will be used last label, value at last position in window
def reshapeData(inputs, outputs, shape, moving_window, mode_label):  

    shape = tuple([x for x in shape if x is not None])

    window = 1

    for dim in shape:
        window = window * dim

    window = int(window / len(inputs[0]))

    if moving_window:

        inputs_reshaped = []

        if mode_label:
            outputs_reshaped = []
        else:
            outputs_reshaped = outputs[window-1:]

        for i in range(window, len(inputs)+1):
            value = inputs[i-window:i, :]
            value = value.reshape(shape)
            inputs_reshaped.append(value)

            if mode_label:
                outputs_reshaped.append(outputs[i-window:i, :].mean(axis=0).round(0))

    else:

        inputs_reshaped = []
        outputs_reshaped = []

        chuncks = int(len(inputs)/window)

        for i in range(0, chuncks):
            value = inputs[i*window : (i+1)*window, :]
            value = value.reshape(shape)
            inputs_reshaped.append(value)

            if mode_label:
                outputs_reshaped.append(outputs[i*window : (i+1)*window, :].mean(axis=0).round(0))
            else:
                outputs_reshaped.append(outputs[((i+1)*window)-1])

    return inputs_reshaped, outputs_reshaped

# Get train and test sets from normalized sets.
# Inputs: normalized sets (dict), sets_train (string[]) and sets_test (string[]) with datasets names ("pvs_x"), shape to reshape data, window type and sides.
# Outputs: df -> input train, input test, output train, output test
def getTrainTestSets(normalized_sets, sets_train, sets_test, shape, moving_window=True, sides=['left', 'right'], mode_label=False):

    input_train = []
    input_test = []
    output_train = []
    output_test = []

    for key in normalized_sets.keys():

        for side in sides:

            inputs, outputs = reshapeData(normalized_sets[key][side].values, normalized_sets[key]["labels"].values, shape, moving_window, mode_label)
        
            if (key in sets_train):

                for inp in inputs:
                    input_train.append(inp)

                for out in outputs:
                    output_train.append(out)             
              
            elif (key in sets_test):

                for inp in inputs:
                    input_test.append(inp)

                for out in outputs:
                    output_test.append(out) 

    return input_train, input_test, output_train, output_test

def createMemoryMap(path, input_train, input_test, output_train, output_test):

    it_file = os.path.join(path, 'input_train.dat')
    it_map = np.memmap(it_file, dtype='float64', mode='w+', shape=((len(input_train),) + input_train[0].shape))
    it_map[:] = input_train[:]

    iv_file = os.path.join(path, 'input_test.dat')
    iv_map = np.memmap(iv_file, dtype='float64', mode='w+', shape=((len(input_test),) + input_test[0].shape))
    iv_map[:] = input_test[:]

    ot_file = os.path.join(path, 'output_train.dat')
    ot_map = np.memmap(ot_file, dtype='float64', mode='w+', shape=((len(output_train),) + output_train[0].shape))
    ot_map[:] = output_train[:]

    ov_file = os.path.join(path, 'output_test.dat')
    ov_map = np.memmap(ov_file, dtype='float64', mode='w+', shape=((len(output_test),) + output_test[0].shape))
    ov_map[:] = output_test[:]

    return it_map, iv_map, ot_map, ov_map

<h5>Model Management</h5>

In [ ]:
def modelFileSavedFormat(file):
    return file + '-train-acc-{acc:.5f}-val-acc-{val_acc:.5f}.hdf5'

def createPathIfNotExists(path):

    if not os.path.exists(path):
        os.makedirs(path)

def saveModelDiagram(model, path, file, show=True):
    createPathIfNotExists(path)
    plot_model(model, to_file=os.path.join(path, file + '.png'), show_shapes=True, show_layer_names=True)

    if show:
        display(plot_model(model, show_shapes=True, show_layer_names=True))
        # print(model.summary())
    
def showHistory(history):
    
    for key in history.history.keys():
        plt.plot(history.history[key], label=key)
    
    plt.legend()
    
def fitModel(model, inputs_train, outputs_train, inputs_validation, outputs_validation, path, file, batch_size=64, epochs=1000, patience=10):
    
    createPathIfNotExists(path)
    
    # train_folder = os.path.join(path, 'train')
    # createPathIfNotExists(train_folder)
    # checkpoint_file_train = os.path.join(train_folder, 'checkpoint-train-{epoch:002d}-{loss:.10f}-{acc:.5f}-val-{val_loss:.10f}-{val_acc:.5f}.hdf5')
    # checkpoint_train = ModelCheckpoint(filepath=checkpoint_file_train, save_best_only=True, monitor='acc', mode='max')

    # validation_folder = os.path.join(path, 'validation')
    # createPathIfNotExists(validation_folder)
    # checkpoint_file_validation = os.path.join(validation_folder, 'checkpoint-{epoch:002d}-train-{loss:.10f}-{acc:.5f}-val-{val_loss:.10f}-{val_acc:.5f}.hdf5')
    # checkpoint_validation = ModelCheckpoint(filepath=checkpoint_file_validation, save_best_only=True, monitor='val_acc', mode='max', verbose=1)

    checkpoint_file_validation = os.path.join(path, modelFileSavedFormat(file))
    checkpoint_validation = ModelCheckpoint(filepath=checkpoint_file_validation, save_best_only=True, monitor='val_acc', mode='max') # verbose=1
    
    # logger_file = os.path.join(path, file + '-training-log.csv')
    # csv_logger = CSVLogger(logger_file, append=True)
    
    plotlosses = PlotLossesKeras()
    
    early_stopping = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=patience, min_delta=0.0001, restore_best_weights=True)
    
    # callbacks=[csv_logger, checkpoint_train, checkpoint_validation, early_stopping]
    callbacks=[plotlosses, checkpoint_validation, early_stopping]
    return model.fit(inputs_train, outputs_train, validation_data=(inputs_validation, outputs_validation), epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=0) # verbose=1

def predictModel(model, inputs):
    return model.predict(inputs)
    
def evaluateModel(model, inputs, outputs, batch_size=64):
    return model.evaluate(inputs, outputs, batch_size=batch_size, verbose=0)
    
def loadWeights(model, pathFile):
    model.load_weights(pathFile)

<h5>Parameter Variations</h5>

In [ ]:
experiment_by_dataset = [
    { "train": ["pvs_1", "pvs_3", "pvs_4", "pvs_6", "pvs_7", "pvs_9"], "test":  ["pvs_2", "pvs_5", "pvs_8"]},
    { "train": ["pvs_1", "pvs_2", "pvs_3", "pvs_7", "pvs_8", "pvs_9"], "test":  ["pvs_4", "pvs_5", "pvs_6"]},
    { "train": ["pvs_1", "pvs_2", "pvs_4", "pvs_6", "pvs_8", "pvs_9"], "test":  ["pvs_3", "pvs_5", "pvs_7"]}
]

experiment_by_fields = [
    getFields(acc=True,  gyro=False, speed=True, below_suspension=True), # acc_below_suspension
    getFields(acc=False, gyro=True,  speed=True, below_suspension=True), # gyro_below_suspension
    getFields(acc=True,  gyro=True,  speed=True, below_suspension=True), # acc_gyro_below_suspension
    getFields(acc=True,  gyro=False, speed=True, above_suspension=True), # acc_above_suspension
    getFields(acc=False, gyro=True,  speed=True, above_suspension=True), # gyro_above_suspension
    getFields(acc=True,  gyro=True,  speed=True, above_suspension=True), # acc_gyro_above_suspension
    getFields(acc=True,  gyro=False, speed=True, dashboard=True), # acc_dashboard
    getFields(acc=False, gyro=True,  speed=True, dashboard=True), # gyro_dashboard
    getFields(acc=True,  gyro=True,  speed=True, dashboard=True) # acc_gyro_dashboard
]

<h5>Labels Fields</h5>

In [ ]:
surface_type_labels = ["dirt_road", "cobblestone_road", "asphalt_road"]

In [ ]:
surface_type_labels_plot = ["Dirt \n Road", "Cobblestone \n Road", "Asphalt \n Road"]  

<h5>Execution Log</h5>

In [ ]:
# Save a log for each experiment execution (params for each execution)
def saveExecutionLog(path, data, columns=['experiment', "window", "scaler", "input_shape", "output_shape", "train_loss", "val_loss", "train_acc", "val_acc"]):
    save = pd.DataFrame(columns=columns, data=data)
    save.to_csv(os.path.join(path, "experiment-execution-log.csv"), index=False)

<h5>Training Functions</h5>

In [ ]:
def manageFiles(history, experiment_folder, experiment_file):

    test = -1
    index = -1
    val_acc = -1
    
    for i in range(0,3):

        max_value = max(history[i]['val_acc'])

        if max_value > val_acc:
            val_acc = max_value
            test = i
            index = history[i]['val_acc'].index(max_value)
            
    train_acc = history[test]['acc'][index]
    train_loss = history[test]['loss'][index]
    val_acc = history[test]['val_acc'][index]
    val_loss = history[test]['val_loss'][index]
    
    test_folder = os.path.join(experiment_folder, "Test " + str(test + 1)) 
    file = modelFileSavedFormat(experiment_file).format(**{'acc': train_acc, 'val_acc': val_acc})

    move_from = os.path.join(test_folder, file)
    move_to = os.path.join(experiment_folder, file)

    shutil.move(move_from, move_to)
    
    for i in range(0,3):
        shutil.rmtree(os.path.join(experiment_folder, "Test " + str(i + 1)))

    return [train_loss, val_loss, train_acc, val_acc]

In [ ]:
def getLoadBar():
    
    global load_bar_main, load_bar_experiment, load_bar_retries, experiment_total_retries
    
    experiment_total = len(experiment_by_dataset)
    experiment_total_iteration = len(input_shapes) * len(scalers)
    experiment_total_retries = 3
    
    load_bar_main = tqdm(total=experiment_total, desc='Main Progress')
    load_bar_experiment = tqdm(total=experiment_total_iteration)
    load_bar_retries = tqdm(total=experiment_total_retries, desc='Retries')

In [ ]:
def run(model_fn, patience=10, batch_size=64, epochs=1000, retries=3, experiments=[0,1,2]):

    load_bar_main.reset()

    for experiment_number in experiments:

        load_bar_experiment.reset()
        load_bar_experiment.set_description("Experiment " + str(experiment_number + 1))

        sets_train = experiment_by_dataset[experiment_number]['train']
        sets_test = experiment_by_dataset[experiment_number]['test']

        execution_log = []

        for input_shape, window_size in input_shapes:

            model_args = parameters(input_shape, output_shape)

            for scaler_function, scaler_args, scaler_name in scalers:

                history = []
                scaler, normalized_sets = scaler_function(subsets) if scaler_args is None else scaler_function(subsets, scaler_args) 
                input_train, input_test, output_train, output_test = getTrainTestSets(normalized_sets, sets_train, sets_test, input_shape, moving_window, sides, mode_label)

                if useTfV2:
                    input_train = np.array(input_train)
                    input_test = np.array(input_test)
                    output_train = np.array(output_train)
                    output_test = np.array(output_test)
                else:
                    input_train = [input_train]
                    input_test = [input_test]
                    output_train = [output_train]
                    output_test = [output_test]

                for test in range(0, retries):

                    model, model_name = model_fn(**model_args)

                    experiment_folder = os.path.join(work_folder, model_name, "Experiment " + str(experiment_number + 1))
                    test_folder = os.path.join(experiment_folder, "Test " + str(test + 1))
                    diagram_file = "experiment-" + str(experiment_number + 1) + "-window-" + str(window_size)
                    experiment_file = diagram_file + "-scaler-" + scaler_name + ("" if scaler_args is None else "-" + str(scaler_args))

                    saveModelDiagram(model, experiment_folder, diagram_file)
                    hist = fitModel(model, input_train, output_train, input_test, output_test, test_folder, experiment_file, patience=patience, batch_size=batch_size, epochs=epochs)

                    history.append(hist.history)
                    load_bar_retries.update(1)

                    # Clean Memory
                    model, hist = [None, None]
                    
                    if not(output is None):
                        output.clear()

                    gc.collect()

                metrics = manageFiles(history, experiment_folder, experiment_file)
                
                execution_log.append([
                    experiment_number + 1, 
                    window_size, 
                    scaler_name + ("" if scaler_args is None else " " + str(scaler_args)), 
                    str(input_shape), 
                    str(output_shape)
                ] + metrics)

                saveExecutionLog(experiment_folder, execution_log)
                load_bar_experiment.update(1)

                # Clean
                history, scaler, normalized_sets = [None, None, None]
                input_train, input_test, output_train, output_test = [None, None, None, None]
                load_bar_retries.reset()
                gc.collect()

        load_bar_main.update(1)       

In [ ]:
def confusionMatrix(files, title, model_fn, input_shape, output_shape, scaler_function, scaler_args):

    matrix = []

    for experiment_number in range(0,3):

        sets_train = experiment_by_dataset[experiment_number]['train']
        sets_test = experiment_by_dataset[experiment_number]['test']

        scaler, normalized_sets = scaler_function(subsets, scaler_args) 
        input_train, input_test, output_train, output_test = getTrainTestSets(normalized_sets, sets_train, sets_test, input_shape, moving_window, sides, mode_label)

        input_train = np.array(input_train)
        input_test = np.array(input_test)
        output_train = np.array(output_train)
        output_test = np.array(output_test)

        model_args = parameters(input_shape, output_shape)
        model, model_name = model_fn(**model_args)
        loadWeights(model, os.path.join(work_folder, model_name, "Experiment " + str(experiment_number + 1), files[experiment_number]))
        predictions = predictModel(model, input_test)
        print(accuracy_score(output_test.argmax(axis=1), predictions.argmax(axis=1)))
        matrix.append(confusion_matrix(output_test.argmax(axis=1), predictions.argmax(axis=1)))

    values = (matrix[0] + matrix[1] + matrix[2])
    con_mat_df = pd.DataFrame(values, index=surface_type_labels_plot, columns=surface_type_labels_plot)
    figure = plt.figure(figsize=(4,4))
    sns.set(font_scale=1.2)
    sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues, fmt='g', annot_kws={"size": 14})
    plt.tight_layout()
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    figure.savefig('confusion_matrix.png', bbox_inches="tight")
    # plt.savefig('results.png')

In [ ]:
def classificationReport(files, model_fn, input_shape, output_shape, scaler_function, scaler_args):

    output_values = []
    predicted_values = []

    for experiment_number in range(0,3):

        sets_train = experiment_by_dataset[experiment_number]['train']
        sets_test = experiment_by_dataset[experiment_number]['test']

        scaler, normalized_sets = scaler_function(subsets, scaler_args) 
        input_train, input_test, output_train, output_test = getTrainTestSets(normalized_sets, sets_train, sets_test, input_shape, moving_window, sides, mode_label)

        input_train = np.array(input_train)
        input_test = np.array(input_test)
        output_train = np.array(output_train)
        output_test = np.array(output_test)

        model_args = parameters(input_shape, output_shape)
        model, model_name = model_fn(**model_args)
        loadWeights(model, os.path.join(work_folder, model_name, "Experiment " + str(experiment_number + 1), files[experiment_number]))
        predictions = predictModel(model, input_test)

        for line in output_test:
            output_values.append(line)

        for line in predictions:
            predicted_values.append(line)

        del input_train, input_test, output_train, output_test

    return classification_report(y_true = np.array(output_values).argmax(axis=1), y_pred = np.array(predicted_values).argmax(axis=1), output_dict=True, target_names=surface_type_labels)